### 线程和进程

1. 进程（Process） = 独立的工厂车间

    - 定义：操作系统分配资源（内存、水电）的基本单位。

    - 特点：每个车间（进程）都有独立的围墙（独立的地址空间）。

    - 隔离性：车间 A 的工人通常不能直接跑到车间 B 去干活，也不能随意拿车间 B 的原料。

    - 通信（IPC）：如果两个车间要交换货物（数据），必须通过专门的通道（管道、套接字等），这比较麻烦，成本较高。

    - 创建：盖新车间（fork/spawn）开销比较大。

2. 线程（Thread） = 车间里的工人

    - 定义：CPU 调度的基本执行单元。

    - 特点：一个车间（进程）里可以有一个或多个工人（线程）。

    - 共享性：同一个车间里的工人们共享车间的所有资源（内存、饮水机、工具）。

    - 通信：工人们面对面工作，传递工具（数据）非常容易，效率极高。

    - 风险：如果一个工人把车间炸了（线程崩溃），整个车间（进程）可能都会停工。

3. 并发 (Concurrency) = 一个人玩杂耍（宏观并行，微观串行）

    - 场景：单核 CPU（只有一个大脑）。

    - 操作：你左手抛球，右手接球，中间还喝口水。

    - 实质：虽然看起来这三件事在同时进行，但实际上在每一个微小的瞬间，你只在做一件事。是因为你切换动作的速度太快了（毫秒级），观众觉得你在同时做。

    - 重点：强调的是**“处理任务的能力”**（能应对多个任务），通过快速切换上下文（Context Switch）实现。

4. 并行 (Parallel) = 多个人一起赛跑（真正的同时）

    - 场景：多核 CPU（有多个大脑）。

    - 操作：你有三只手（或者三个人），一只手抛球，一只手接球，一只手拿水杯。

    - 实质：在同一个时刻，这几件事真的都在发生。

    - 重点：强调的是**“同时执行的能力”**，必须依赖硬件支持（多核）。

### 多线程编程

Python 标准库中threading模块的Thread类可以帮助我们非常轻松的实现多线程编程。我们用一个联网下载文件的例子来对比使用多线程和不使用多线程到底有什么区别，代码如下所示。

不使用多线程的下载。

In [1]:
import random
import time

def download(filename):
    start = time.time()
    print(f"Downloading {filename}...")
    time.sleep(random.randint(3, 6))
    print(f"Downloaded {filename}")
    end = time.time()
    print(f"Download time: {end - start:.2f} seconds")

def main():
    start = time.time()
    download('MySQL从删库到跑路.avi')
    download('Python从入门到住院.pdf')
    end = time.time()
    print(f"Total time: {end - start:.2f} seconds")

if __name__ == "__main__":
    main()

Downloaded MySQL从删库到跑路.avi
Download time: 5.00 seconds
Downloaded Python从入门到住院.pdf
Download time: 5.01 seconds
Total time: 10.01 seconds


可以看出，当我们的程序只有一个工作线程时，每个下载任务都需要等待上一个下载任务执行结束才能开始，所以程序执行的总耗时是两个下载任务各自执行时间的总和。

事实上，上面的两个下载任务之间并没有逻辑上的因果关系，两者是可以“并发”的，下一个下载任务没有必要等待上一个下载任务结束，为此，我们可以使用多线程编程来改写上面的代码。

In [ ]:
import random
import time
from threading import Thread

def download(*, filename):
    start = time.time()
    print(f'开始下载 {filename}.')
    time.sleep(random.randint(3, 6))
    print(f'{filename} 下载完成.')
    end = time.time()
    print(f'下载耗时: {end - start:.3f}秒.')
    
def main():
    threads = [
        Thread(target=download, kwargs={'filename': 'Python从入门到住院.pdf'}),
        Thread(target=download, kwargs={'filename': 'MySQL从删库到跑路.avi'}),
        Thread(target=download, kwargs={'filename': 'Linux从精通到放弃.mp4'})
    ]
    start = time.time()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    end = time.time()
    print(f"Total time: {end - start:.2f} seconds")

if __name__ == "__main__":
    main()

Downloaded MySQL从删库到跑路.avi
Download time: 3.00 seconds
Downloaded Linux从精通到放弃.mp4
Download time: 4.00 seconds
Downloaded Python从入门到住院.pdf
Download time: 5.00 seconds
Total time: 5.00 seconds


整个程序的执行时间几乎等于耗时最长的一个下载任务的执行时间，这也就意味着，三个下载任务是并发执行的，不存在一个等待另一个的情况，这样做很显然提高了程序的执行效率。

### 使用 Thread 类创建线程对象

通过上面的代码可以看出，直接使用Thread类的构造器就可以创建线程对象，而线程对象的start()方法可以启动一个线程。线程启动后会执行target参数指定的函数，当然前提是获得 CPU 的调度；如果target指定的线程要执行的目标函数有参数，需要通过args参数为其进行指定，对于关键字参数，可以通过kwargs参数进行传入。Thread类的构造器还有很多其他的参数，我们遇到的时候再为大家进行讲解，目前需要大家掌握的，就是target、args和kwargs。

### 继承 Thread 类自定义线程

除了上面的代码展示的创建线程的方式外，还可以通过继承Thread类并重写run()方法的方式来自定义线程，具体的代码如下所示。

In [4]:
import random
import time
from threading import Thread

class DownloadThread(Thread):
    def __init__(self, filename):
        self.filename = filename
        super().__init__()
    
    def run(self):
        start = time.time()
        print(f"Downloading {self.filename}...")
        time.sleep(random.randint(3, 6))
        print(f"Downloaded {self.filename}")
        end = time.time()
        print(f"Download time for {self.filename}: {end - start:.2f} seconds")

def main():
    threads = [
        DownloadThread('Python从入门到住院.pdf'),
        DownloadThread('MySQL从删库到跑路.avi'),
        DownloadThread('Linux从精通到放弃.mp4')
    ]
    start = time.time()
    # 启动三个线程
    for thread in threads:
        thread.start()
    # 等待线程结束
    for thread in threads:
        thread.join()
    end = time.time()
    print(f'总耗时: {end - start:.3f}秒.')


if __name__ == '__main__':
    main()


Downloaded MySQL从删库到跑路.avi
Download time for MySQL从删库到跑路.avi: 3.01 seconds
Downloaded Python从入门到住院.pdf
Download time for Python从入门到住院.pdf: 5.01 seconds
Downloaded Linux从精通到放弃.mp4
Download time for Linux从精通到放弃.mp4: 6.01 seconds
总耗时: 6.007秒.


### 使用线程池

我们还可以通过线程池的方式将任务放到多个线程中去执行，通过线程池来使用线程应该是多线程编程最理想的选择。事实上，线程的创建和释放都会带来较大的开销，频繁的创建和释放线程通常都不是很好的选择。利用线程池，可以提前准备好若干个线程，在使用的过程中不需要再通过自定义的代码创建和释放线程，而是直接复用线程池中的线程。Python 内置的concurrent.futures模块提供了对线程池的支持，代码如下所示。

In [5]:
import random
import time
from concurrent.futures import ThreadPoolExecutor
from threading import Thread


def download(*, filename):
    start = time.time()
    print(f'开始下载 {filename}.')
    time.sleep(random.randint(3, 6))
    print(f'{filename} 下载完成.')
    end = time.time()
    print(f'下载耗时: {end - start:.3f}秒.')


def main():
    with ThreadPoolExecutor(max_workers=4) as pool:
        filenames = ['Python从入门到住院.pdf', 'MySQL从删库到跑路.avi', 'Linux从精通到放弃.mp4']
        start = time.time()
        for filename in filenames:
            pool.submit(download, filename=filename)
    end = time.time()
    print(f'总耗时: {end - start:.3f}秒.')


if __name__ == '__main__':
    main()

开始下载 Python从入门到住院.pdf.
开始下载 MySQL从删库到跑路.avi.
开始下载 Linux从精通到放弃.mp4.
Linux从精通到放弃.mp4 下载完成.MySQL从删库到跑路.avi 下载完成.
下载耗时: 4.006秒.

下载耗时: 4.006秒.
Python从入门到住院.pdf 下载完成.
下载耗时: 6.006秒.
总耗时: 6.008秒.


### 守护线程

在主线程结束的时候，不值得再保留的执行线程。这里的不值得保留指的是守护线程会在其他非守护线程全部运行结束之后被销毁，它守护的是当前进程内所有的非守护线程。简单的说，守护线程会跟随主线程一起挂掉，而主线程的生命周期就是一个进程的生命周期。如果不理解，我们可以看一段简单的代码。

In [ ]:
import time
from threading import Thread

def display(content):
    while True:
        print(content, end='', flush=True)
        time.sleep(0.1)

def main():
    Thread(target=display, args=('Ping', )).start()
    Thread(target=display, args=('Pong', )).start()

if __name__ == '__main__':
    main()

上面的代码运行起来之后是不会停止的，因为两个子线程中都有死循环，除非你手动中断代码的执行。但是，如果在创建线程对象时，将名为daemon的参数设置为True，这两个线程就会变成守护线程，那么在其他线程结束时，即便有死循环，两个守护线程也会挂掉，不会再继续执行下去，代码如下所示。

In [ ]:
import time
from threading import Thread

def display(content):
    while True:
        print(content, end='', flush=True)
        time.sleep(0.1)

def main():
    Thread(target=display, args=('Ping', ), daemon=True).start()
    Thread(target=display, args=('Pong', ), daemon=True).start()
    time.sleep(5)

if __name__ == '__main__':
    main()